## Import all required libraries

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df
from message_ix.tools.add_dac import *

%matplotlib inline

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


In [2]:
import ixmp
import message_ix

from message_data.tools.post_processing.iamc_report_hackathon import (
    report as reporting)

import warnings
warnings.filterwarnings('ignore')

setups = {#'SSP_SSP2_v1.0':'NPi2030_600f',
          #'SSP_SSP5_v1.0':'NPi2030_700f',
          #'SSP_SSP2_v1.0':'SSP2 - Low Overshoot_Relaxed-CCS',
          #'SSP_SSP4_v1.0':'SSP4 - Low Overshoot_Relaxed-CCS',
          #'SSP_SSP5_v1.0':'SSP5 - Low Overshoot_Relaxed-CCS-v2',
          #'SSP_SSP5_v1.0':'SSP5 - Low Overshoot_Relaxed-CCS-v4',
          'SSP_SSP5_v1.0':'SSP5 - Low Overshoot_Relaxed-CCS-v5',
          }

for key,val in setups.items():
    print(key,'//',val)
    mp = ixmp.Platform()
    mo_name = key
    sc_name = val
    scen = message_ix.Scenario(mp, model=mo_name, scenario=sc_name)

    # report(scenario)
    print("Reporting standard variables")
    reporting(
        mp,
        scen,
        # NB(PNK) this is not an error; .iamc_report_hackathon.report() expects a
        #         string containing "True" or "False" instead of an actual bool.
        "False",
        scen.model,
        scen.scenario,
        merge_hist=True,
        merge_ts=True,
        run_config="materials_daccs_run_config.yaml",
    )
    mp.close_db()

SSP_SSP5_v1.0 // SSP5 - Low Overshoot_Relaxed-CCS-v5
Reporting standard variables
processing Table: Resource|Extraction
processing Table: Resource|Cumulative Extraction
processing Table: Primary Energy
processing Table: Primary Energy (substitution method)
processing Table: Final Energy
processing Table: Secondary Energy|Electricity
processing Table: Secondary Energy|Heat
processing Table: Secondary Energy
processing Table: Secondary Energy|Gases
processing Table: Secondary Energy|Solids
processing Table: Emissions|CO2
The difference between top-down and bottom-up accounting.
processing Table: Emissions|BC
processing Table: Emissions|OC
processing Table: Emissions|CO
processing Table: Emissions|N2O
processing Table: Emissions|CH4
processing Table: Emissions|NH3
processing Table: Emissions|Sulfur
processing Table: Emissions|NOx
processing Table: Emissions|VOC
processing Table: Emissions|HFC
processing Table: Emissions
processing Table: Emissions
processing Table: Agricultural Demand
pro